In [1]:
import cv2
import numpy as np
import os
import pandas
import sys
import keras as K
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Add
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight


Using TensorFlow backend.


In [2]:
def read_images(filenames,directory,height,width):
    # Read Images into a Numpy array
    
    # Get the array size
    array_size = len(filenames)
    images = np.array([])
    #Preallocate the array.  This is much faster that numpy.append
    images = np.zeros((array_size,height,width))

    i=0
    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Resize the image.
        resized_image = cv2.resize(image, (width,height))
        # Add it to numpy array
        images[i] = resized_image
        i+=1

    
    
    return images

In [3]:
def get_labels(filelist,labels):
    
    labels.index = labels['Image']
    select_labels = np.array([])
    labels_filenames = []
    for file in filelist:
        select_labels = np.append(select_labels,labels.loc[file]['Encoded_Id'])
        labels_filenames.append(file)
    
    return select_labels,labels_filenames

In [4]:
def encode_labels(labels_df):

    labels = labels_df['Id'].values
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_encode = le.transform(labels)
    
    labels_df['Encoded_Id'] = labels_encode 
    
    return labels_df

In [5]:
width = 128
height = 128


traindir="/home/james/Kaggle/Whale/Data/train/augment"
labelsfile="/home/james/Kaggle/Whale/Data/train.csv"

In [6]:
labels = pandas.read_csv(labelsfile)
labels.describe()
with pandas.option_context('display.max_rows', None, 'display.max_columns', None):
    print(labels['Id'].value_counts())

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
w_700ebb4      57
w_3de579a      54
w_564a34b      51
w_fd3e556      50
w_88e4537      49
w_2b069ba      48
w_d405854      47
w_f0fe284      45
w_789c969      45
w_5e8e218      40
w_778e474      40
w_343f088      40
w_5a2634c      37
w_a9304b9      37
w_60ce6fc      37
w_6822dbc      36
w_af367c3      35
w_f765256      34
w_1ca9ab1      34
w_17b0d3a      33
w_d72771c      32
w_8c25681      31
w_08630fd      31
w_6cda039      31
w_fd6a3ad      30
w_51fc1fc      30
w_04003e9      30
w_59052ad      29
w_2365d55      29
w_6caff0f      27
w_91cc02c      25
w_bfcad53      25
w_0a155b9      24
w_0bc078c      24
w_27597ff      24
w_75f6ffa      24
w_cd4cb49      24
w_b9e5911      24
w_990921b      24
w_242fb46      23
w_e1ffbe2      23
w_0135f5f      23
w_5773c71      23
w_efbdcbc      22
w_f61b957      22
w_8da30ad      22
w_488865e      22
w_60cf87c      22
w_4516ff1      21
w_ef83760      21
w_3815890 

In [7]:

labels = pandas.read_csv(labelsfile)

y=labels['Id']
values = np.array(y)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(values),
                                                 values)

#Create Labels

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
train_labels = onehot_encoded


(25361, 5005)


In [8]:
dropout = 0.25
    
# Get number of classes
num_classes = train_labels.shape[1]
input_shape=(height,width,1)
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(128, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(256, (3, 3), activation='relu',padding='same'))
model.add(Dropout(dropout))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=["accuracy"],
              #metrics=[mean_pred]
             )


#Save Models
filepath="/home/james/Kaggle/Whale/Data/whales-copy" + ".best.hdf5"

checkpoint = ModelCheckpoint(filepath, 
                             monitor="acc",
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')


callbacks_list = [checkpoint]


In [9]:
dirnames = os.listdir(traindir)
dirnames.sort()

directories = []
for i in range(20):
    for dir in dirnames:
        directories.append(dir)


i = 0
for directory in directories:
    print("Directory:  ", directory)
    print ("Iteration: ", i)
    
    filenames = os.listdir(traindir + "/" + directory)
    filenames.sort()
    
    chunk_images = read_images(filenames,traindir + "/" + directory,height,width)

    # Reshape Array
    chunk_images = chunk_images.reshape((chunk_images.shape[0],height,width,1))
    
    history = model.fit(chunk_images,train_labels,
                    #X_train, y_train,
                    epochs=4,
                    callbacks=callbacks_list,
                    class_weight=class_weights,
                    #validation_data=(X_test,y_test),
                    batch_size=50,
                    verbose=1,
                    shuffle=True)
    del chunk_images
    i+=1

Directory:   1
Iteration:  0
Epoch 1/4
25361/25361 [==============================] - 48s 2ms/step - loss: 6.4155 - acc: 0.3396

Epoch 00001: acc improved from -inf to 0.33962, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 2/4
25361/25361 [==============================] - 46s 2ms/step - loss: 5.7439 - acc: 0.3810

Epoch 00002: acc improved from 0.33962 to 0.38102, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 3/4
25361/25361 [==============================] - 46s 2ms/step - loss: 5.7393 - acc: 0.3805

Epoch 00003: acc did not improve from 0.38102
Epoch 4/4
25361/25361 [==============================] - 46s 2ms/step - loss: 5.8428 - acc: 0.3794

Epoch 00004: acc did not improve from 0.38102
Directory:   2
Iteration:  1
Epoch 1/4
25361/25361 [==============================] - 45s 2ms/step - loss: 6.0399 - acc: 0.3709

Epoch 00001: acc did not improve from 0.38102
Epoch 2/4
25361/25361 [==============================] - 45s 2ms/step

KeyboardInterrupt: 

In case we want to do validation training later on, we will use a function in Scikit Learn to divide the datasets up.

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(chunk_images,train_labels,
#                                                    test_size=0.33, 
#                                                    random_state=42,
#                                                    shuffle=True)
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

Now that the data is in the proper format and the model has been compiled, the model can be fit.  One can see that the class_weights that were set above have been supplied as an argument to the fit function.  Also notice that the callbacks argument has been initialized to the list that was set above.

In [12]:
testdir = "/home/james/Kaggle/Whale/Data/test/4"
testfilenames = os.listdir(testdir)
testfilenames.sort()
testfiles = []

for testfilename in testfilenames:
    testfiles.append(testdir + "/" + testfilename)
    
test_images = read_images(testfiles,"",height,width)
# Reshape Array
test_images = test_images.reshape((test_images.shape[0],height,width,1))


In [13]:
model.load_weights(filepath)
# Make prediction
Predictions = model.predict(test_images)

In [14]:
test_csv = "/home/james/Kaggle/Whale/Data/test.csv"

test = testfilenames
col = ['Image']
test_df = pandas.DataFrame(test, columns=col)
test_df['Id'] = ''

for i, pred in enumerate(Predictions):
    ids = label_encoder.inverse_transform(pred.argsort()[-5:][::-1])
    ids = ids.tolist()
    sorted_idx = pred.argsort()[-5:][::-1]
    sorted_values = []
    for idx in sorted_idx:
        sorted_values.append(pred[idx])

    print(sorted_values)
    
    test_df.loc[i, 'Id'] = ' '.join(ids)

print(test_df.head(10))

test_df.to_csv(test_csv, index=False)

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.69494, 0.104041815, 0.04999823, 0.022061609, 0.02091325]
[0.67750984, 0.19052882, 0.025634972, 0.014885649, 0.014477892]
[0.3013774, 0.15069552, 0.14585038, 0.08187558, 0.055807568]
[0.5788694, 0.045042407, 0.029841706, 0.021227863, 0.019932779]
[0.8424881, 0.03415388, 0.027038537, 0.01819284, 0.0053640557]
[0.7520551, 0.13113916, 0.012630674, 0.01229476, 0.007977461]
[0.33343163, 0.18315555, 0.091112286, 0.07042166, 0.04287087]
[0.8791729, 0.012809066, 0.009057124, 0.008035961, 0.007948083]
[0.6700049, 0.04957554, 0.03506467, 0.03235512, 0.016155511]
[0.7931922, 0.0140849585, 0.013503324, 0.008887067, 0.008673056]
[0.72807086, 0.05974631, 0.022326082, 0.020668484, 0.01633942]
[0.21708742, 0.1652557, 0.06002828, 0.034435395, 0.028160406]
[0.82814753, 0.041815598, 0.024610763, 0.011501451, 0.010172365]
[0.67302567, 0.052417412, 0.027826494, 0.021513622, 0.020351233]
[0.5743898, 0.081195496, 0.05897079, 0.03620188, 0.03387047]
[0.73239607, 0.042417653, 0.038315453, 0.030377965, 0.0238

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.36700577, 0.15071097, 0.06371263, 0.034962084, 0.027240343]
[0.91616565, 0.021142432, 0.0077288947, 0.0069508953, 0.0049778484]
[0.8501203, 0.014839727, 0.014465998, 0.00653918, 0.006380954]
[0.6711987, 0.07259046, 0.037789237, 0.020716097, 0.014200253]
[0.7493677, 0.06223642, 0.04717856, 0.02039752, 0.011105735]
[0.5833921, 0.17581554, 0.06320645, 0.024873832, 0.018631114]
[0.35636553, 0.24185029, 0.04479681, 0.021504588, 0.01992656]
[0.31409732, 0.29243603, 0.049931716, 0.020447759, 0.02026508]
[0.4110597, 0.33507448, 0.09362249, 0.03909116, 0.03037703]
[0.56030166, 0.09975562, 0.01128976, 0.009582555, 0.008513055]
[0.7895542, 0.04279947, 0.034797017, 0.028558949, 0.013030377]
[0.25729877, 0.080311604, 0.07892305, 0.05998357, 0.05196138]
[0.6837327, 0.06301919, 0.04091565, 0.023651002, 0.021552276]
[0.6032044, 0.20203312, 0.071409486, 0.020636562, 0.0107622575]
[0.52346516, 0.05529176, 0.04279814, 0.035394557, 0.026858406]
[0.9609806, 0.0128609715, 0.0060274308, 0.004286085, 0.003

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

...

[0.95811313, 0.026476616, 0.0028565736, 0.0020686558, 0.0020003146]
[0.5023109, 0.1843935, 0.087814026, 0.049032763, 0.022356465]
[0.98757195, 0.00300962, 0.0027276601, 0.0011171972, 0.0005261026]
[0.7010252, 0.05707847, 0.042159338, 0.026309272, 0.021482568]
[0.30421835, 0.084430866, 0.07027067, 0.04074338, 0.038717397]
[0.49383673, 0.14235729, 0.07954585, 0.055867553, 0.026803577]
[0.3394905, 0.083145276, 0.03600446, 0.028006526, 0.02459444]
[0.23366438, 0.11147306, 0.10483556, 0.046484135, 0.040288635]
[0.8452138, 0.032728642, 0.014543747, 0.012563861, 0.0070166304]
[0.24051255, 0.20245269, 0.19733013, 0.052997448, 0.045394976]
[0.3729721, 0.29776025, 0.05360096, 0.022283854, 0.022079064]
[0.32012618, 0.2383854, 0.046671256, 0.04038645, 0.031218406]
[0.30511388, 0.18310875, 0.10684998, 0.039074473, 0.038084373]
[0.8179847, 0.052834343, 0.011082699, 0.009717626, 0.0072388877]
[0.68335086, 0.06458779, 0.03024736, 0.022531372, 0.019648088]
[0.34296486, 0.2248058, 0.109261096, 0.0892373

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.45354927, 0.26138556, 0.03674683, 0.029364258, 0.02559902]
[0.7264076, 0.04939293, 0.020355387, 0.019161573, 0.01861785]
[0.61536825, 0.094208956, 0.017319463, 0.015961435, 0.015889008]
[0.6053648, 0.06939347, 0.042355947, 0.04081093, 0.017174974]
[0.9606285, 0.019199355, 0.0031026457, 0.0011137525, 0.00076657726]
[0.700334, 0.10837705, 0.015979702, 0.015481116, 0.013359291]
[0.42191696, 0.14578113, 0.039376426, 0.024713412, 0.023967935]
[0.29909918, 0.13715924, 0.056439478, 0.037853096, 0.029964948]
[0.956661, 0.0036914463, 0.0033000722, 0.002372022, 0.00226]
[0.47296563, 0.16565938, 0.107672036, 0.03150409, 0.031324968]
[0.14217384, 0.13472384, 0.11987638, 0.10244422, 0.05974253]
[0.9873998, 0.0041847993, 0.0012062103, 0.000830214, 0.00080499455]
[0.36025164, 0.13841599, 0.026107246, 0.017216451, 0.01441537]
[0.43046802, 0.22249967, 0.21567877, 0.041406646, 0.009454599]
[0.89963675, 0.009204397, 0.0087878825, 0.008238188, 0.007358765]
[0.9689716, 0.0065499586, 0.0010716749, 0.0008

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.6836642, 0.028255297, 0.016925417, 0.014015482, 0.010966102]
[0.5335216, 0.07643092, 0.07393991, 0.06013795, 0.03637838]
[0.722033, 0.043956637, 0.022351429, 0.020577006, 0.016759329]
[0.30803743, 0.25303304, 0.09038578, 0.049075346, 0.048305612]
[0.62334466, 0.058343224, 0.055057123, 0.024156857, 0.02169032]
[0.79769224, 0.02013212, 0.013944502, 0.0068195616, 0.005622001]
[0.8547263, 0.031980284, 0.030407095, 0.006823173, 0.0037139743]
[0.775347, 0.08024106, 0.029904049, 0.020168109, 0.009108707]
[0.34760544, 0.2133666, 0.09996422, 0.09891631, 0.017556474]
[0.7699213, 0.02239807, 0.019831406, 0.018841505, 0.01800271]
[0.8781887, 0.01686238, 0.0123051535, 0.008913629, 0.00745141]
[0.519675, 0.117986485, 0.044661153, 0.02960069, 0.028481342]
           Image                                                 Id
0  00028a005.jpg  new_whale w_6fe5b8e w_04003e9 w_76ad1d2 w_fdd3687
1  000dcf7d8.jpg  w_d72771c new_whale w_f5bf6df w_98f2705 w_02aa597
2  000e7c7df.jpg  new_whale w_53215c6 w_cd